In [ ]:
!pip install --no-index --find-links=/kaggle/input/ariel-2024-pqdm pqdm
# !pip install astropy

In [ ]:
import numpy as np
import pandas as pd
from astropy.stats import sigma_clip
import os 
import pandas as pd
import pandas.api.types
import scipy.stats
from tqdm import tqdm
import itertools
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import plotly.express as px
import matplotlib.pyplot as plt 
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy


In [ ]:
from scipy.signal import savgol_filter

def smooth_data(data, window_size):
    return savgol_filter(data, window_size, 3)

def optimize_breakpoint(data, initial_breakpoint, window_size, buffer_size, smooth_window):
    best_breakpoint = initial_breakpoint
    best_score = float("-inf")
    midpoint = len(data) // 2
    smoothed_data = smooth_data(data, smooth_window)
#     smoothed_data=data
    for i in range(-window_size, window_size):
        new_breakpoint = initial_breakpoint + i
        if new_breakpoint > buffer_size and new_breakpoint < midpoint - buffer_size:
            region1 = data[: new_breakpoint - buffer_size]
            region2 = data[
                new_breakpoint
                + buffer_size : 2 * midpoint
                - new_breakpoint
                - buffer_size
            ]
            region3 = data[2 * midpoint - new_breakpoint + buffer_size :]

            breakpoint_region1 = smoothed_data[new_breakpoint - buffer_size: new_breakpoint + buffer_size]
            breakpoint_region2 = smoothed_data[new_breakpoint - buffer_size: new_breakpoint + buffer_size]

            mean_diff = abs(np.mean(region1) - np.mean(region2)) + abs(
                np.mean(region2) - np.mean(region3)
            )
            var_sum = np.var(region1) + np.var(region2) + np.var(region3)
            range_at_breakpoint1 = (np.max(breakpoint_region1) - np.min(breakpoint_region1))
            range_at_breakpoint2 = (np.max(breakpoint_region2) - np.min(breakpoint_region2))

            mean_range_at_breakpoint = (range_at_breakpoint1 + range_at_breakpoint2) / 2

            score = mean_diff - 0.5 * var_sum + mean_range_at_breakpoint

            if score > best_score:
                best_score = score
                best_breakpoint = new_breakpoint

                
    return best_breakpoint

In [ ]:
def predict_spectra(signal, phase1, phase2, power):
    def objective_to_minimize(s, power, delta=2):
        x = list(range(len(signal) - delta * 4))
        
        y = (
            signal[: phase1 - delta].tolist()
            + (signal[phase1 + delta : phase2 - delta] * (1 + s)).tolist()
            + signal[phase2 + delta :].tolist()
        )
        
        z = np.polyfit(x, y, deg=power)
        p = np.poly1d(z)
        error = np.abs(p(x) - y).mean()
        
        return error
    
    if len(signal.shape) > 1:
        signal = signal.mean(axis=1)
    
    result = minimize(
        fun=lambda s: objective_to_minimize(s[0], power),
        x0=[0.0001],
        method="Nelder-Mead",
        options={"maxiter": 500}
    )
            
    return result.x[0]

In [ ]:
def apply_linear_corr(signal, linear_corr):
    """Apply linear correction to the signal"""
    linear_corr = np.flip(linear_corr, axis=0)
    corrected_signal = signal.copy()
    
    for x, y in np.ndindex(signal.shape[1:]):
        poli = np.poly1d(linear_corr[:, x, y])
        corrected_signal[:, x, y] = poli(signal[:, x, y])
    return corrected_signal

def clean_dark(signal, dark):
    """Clean dark frame from signal"""
    # Correct timing pattern for AIRS
    dt = np.ones(len(signal)) * 0.1
    dt[1::2] += 4.5  # Alternate frames have different exposure time
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    return signal - dark * dt[:, np.newaxis, np.newaxis]

def process_signal(signal, gain, offset, dead, flat, dark, linear_corr):
    """Process AIRS signal with all calibration steps"""
    cut_inf, cut_sup = 39, 321
    
    # Reshape signal to correct dimensions if needed
    if signal.ndim != 3:
        signal = signal.reshape(11250, 32, 356)
    
    # Convert to physical units
    signal = signal / gain + offset
    signal = signal.clip(0)  # Remove negative values
    
    # Apply wavelength cuts
    signal = signal[:, :, cut_inf:cut_sup]
    linear_corr = linear_corr[:, :, cut_inf:cut_sup]
    dark = dark[:, cut_inf:cut_sup]
    dead = dead[:, cut_inf:cut_sup]
    flat = flat[:, cut_inf:cut_sup]
    
    # Apply linear correction
    signal = apply_linear_corr(signal, linear_corr)
    
    # Clean dark frame
    signal = clean_dark(signal, dark)
    
    # Handle dead pixels and hot pixels
    hot = sigma_clip(dark, sigma=5, maxiters=5).mask
    flat = flat.reshape(1, 32, flat.shape[-1])
    flat[dead.reshape(1, 32, -1)] = np.nan
    flat[hot.reshape(1, 32, -1)] = np.nan
    
    # Apply flat field correction
    signal = signal / flat
    
    # Take center pixels (10:22)
    signal = signal[:, 10:22, :]
    
    return signal

def apply_cds(signal):
    """Apply Correlated Double Sampling (odd-even subtraction)"""
    # Average over spatial dimension first
    mean_signal = np.nanmean(signal, axis=1)
    # Subtract even from odd frames
    cds_signal = mean_signal[1::2] - mean_signal[0::2]
    return cds_signal

def bin_signal(signal, binning=30):
    """Bin the signal in time dimension"""
    n_points = signal.shape[0] 
    n_bins = n_points // binning
    
    binned = np.zeros((n_bins, signal.shape[1]))
    for j in range(n_bins):
        start_idx = j * binning
        end_idx = start_idx + binning
        binned[j] = np.nanmean(signal[start_idx:end_idx], axis=0)
    
    return binned

In [ ]:
mode="test"
files= sorted(os.listdir(f"/kaggle/input/ariel-data-challenge-2024/{mode}"))
planets=[]

for f in files:
    planets.append(int(f))
planets.sort()
planets=planets[:]
print(len(planets))
print(planets[0])

In [ ]:
count = 0
adc_info = pd.read_csv(f'/kaggle/input/ariel-data-challenge-2024/{mode}_adc_info.csv')
planet_scales=[]


initial_breakpoint=60
buffer_size=5
smooth_window=10
window_size=5

for planet in planets:
    print(f"\nProcessing planet: {planet}")
    planet = int(planet) 
    
    # Load calibration data
    airs_dark = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{mode}/{planet}/AIRS-CH0_calibration/dark.parquet').values.reshape(32, 356)
    airs_dead = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{mode}/{planet}/AIRS-CH0_calibration/dead.parquet').values.reshape(32, 356)    
    airs_flat = pd.read_parquet( f'/kaggle/input/ariel-data-challenge-2024/{mode}/{planet}/AIRS-CH0_calibration/flat.parquet').values.reshape(32, 356)
    airs_linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{mode}/{planet}/AIRS-CH0_calibration/linear_corr.parquet').values.reshape(6, 32, 356)
    
    # Load signal data
    airs_ch0 = pd.read_parquet( f'/kaggle/input/ariel-data-challenge-2024/{mode}/{planet}/AIRS-CH0_signal.parquet').values.reshape(11250, 32, 356)
    
    selected_adc_info = adc_info.loc[adc_info['planet_id'] == planet]
    airs_ch0_gain = selected_adc_info['AIRS-CH0_adc_gain'].values[0]
    airs_ch0_offset = selected_adc_info['AIRS-CH0_adc_offset'].values[0]
    
    processed = process_signal(airs_ch0, airs_ch0_gain, airs_ch0_offset,  airs_dead, airs_flat, airs_dark, airs_linear_corr)
    
    cds_signal = apply_cds(processed)
    binned_signal = bin_signal(cds_signal, binning=30)
            
    af= binned_signal
    af = np.array(af[ : , :])
    af = np.nanmean(af , axis=(1 ))

    
    airsbp = optimize_breakpoint(af,initial_breakpoint,window_size=window_size,buffer_size=buffer_size,smooth_window=smooth_window)
    midpoint1 = len(af) // 2
    bp1 = [airsbp, 2 * midpoint1 - airsbp]
    ingress_start   =  bp1[0] - buffer_size
    ingress_end     =  bp1[0] + buffer_size
    egress_start =  bp1[1] - buffer_size
    egress_end   =  bp1[1] + buffer_size
    
    breakpoints = [ingress_start, ingress_end, egress_start, egress_end ]

    signal_data=af 
    
    a,b,c,d = breakpoints 
    p_scale=predict_spectra(signal_data, (a+b)//2 , (c+d)//2   , power=3  )
    
    planet_scales.append(p_scale)
        
    
    count += 1
    print(f"saved planet number {planet}", f"total {count}", binned_signal.shape)
print("done")

In [ ]:
# print(planet_scales)

In [ ]:
wave_scales = np.load("/kaggle/input/arieldata/wavescales_power3_spline.npy")

In [ ]:
final=[]

for ps in planet_scales:
    
    wavelengths=[]
    for ws in wave_scales:
        wavelengths.append(ps * ws)
        
    wavelengths=np.abs(np.array(wavelengths))
    final.append(wavelengths)
    

In [ ]:
print(final[0][:5])

In [ ]:
predictions = np.array(final)
columns = ['planet_id'] + [f'wl_{i+1}' for i in range(283)] + [f'sigma_{i+1}' for i in range(283)]
submit_df = pd.DataFrame(columns=columns)

vals=planets

uncertainity = 6e-4
print(uncertainity)
rows_list = []  

for i in range(len(vals)):  
    row_data = {
        'planet_id': vals[i]
    }
    
    for j in range(283):
        row_data[f'wl_{j+1}'] = predictions[i, j]
    
    for j in range(283):
        row_data[f'sigma_{j+1}'] = uncertainity
    
    rows_list.append(row_data)

submit_df = pd.DataFrame(rows_list)

submit_df.to_csv("submission.csv", index=False)
print("saved")